In [1]:
%pip install simpy


In [2]:
# Ejercicio de un Call Center.

# En un call center trabajan 5 empleados que reciben llamadas.
# El sistema comienza con 10 llamadas que están intentando comunicarse con el call center.
# A partir de allí, comienzan a ingresar llamadas entre 1 y 3 minutos de forma equiprobable.
# Cada llamada, una vez que logra ser atendida, tiene una duración de un tiempo que responde a una
# distribución normal de media 5 minutos y desvío estandar de 4 minutos.
# Además, este tiempo no puede ser menor  que 1 minuto ya que  es el tiempo en que
# el operador valida los datos de la llamada.
# Simular durante 2 horas.

import random
import simpy
import numpy as np


In [3]:
CANT_EMPLEADOS = 5
TIEMPO_ATENCION = 300
TIEMPO_ARRIBO = 120
TIEMPO_SIMULACION = 7200

clientesAtendidos = 0


In [4]:
class CallCenter:

    def __init__ (self, env, cantEmpleados, tiempoAtencion):
      self.env = env
      self.staff = simpy.Resource(env, cantEmpleados)
      self.tiempoAtencion = tiempoAtencion

    def atender(self, cliente):
      tiempoAtencionRandom = max(60, np.random.normal(self.tiempoAtencion, 240))
      yield self.env.timeout(tiempoAtencionRandom)
      print(f'Atencion finalizada para el cliente {cliente} a las {self.env.now:.0f}')


In [5]:
def customer(env, nombre, callCenter):

  global clientesAtendidos
  print(f'Cliente {nombre} ingresa a la fila a las {env.now}')
  with callCenter.staff.request() as request:
    yield request
    print(f'Cliente {nombre} es atendido a las {env.now:.0f}')
    yield env.process(callCenter.atender(nombre))
    print(f'Cliente {nombre} termina su atención a las {env.now:.0f}')
    clientesAtendidos += 1


In [6]:
def setup(env, cantEmpleados, tiempoAtencion, tiempoArribo):
  callCenter = CallCenter(env, cantEmpleados, tiempoAtencion)

  for i in range(1,11):
    env.process(customer(env, i, callCenter))

  while True:
    yield env.timeout(random.randint(tiempoArribo-60, tiempoArribo+60))
    i+=1
    env.process(customer(env, i, callCenter))


In [7]:
print('Comenzando la simulacion del Call Center')
env = simpy.Environment()
env.process(setup(env, CANT_EMPLEADOS, TIEMPO_ATENCION, TIEMPO_ARRIBO))
env.run(until=TIEMPO_SIMULACION)

print('Clientes atendidos:', clientesAtendidos)

Comenzando la simulacion del Call Center
Cliente 1 ingresa a la fila a las 0
Cliente 2 ingresa a la fila a las 0
Cliente 3 ingresa a la fila a las 0
Cliente 4 ingresa a la fila a las 0
Cliente 5 ingresa a la fila a las 0
Cliente 6 ingresa a la fila a las 0
Cliente 7 ingresa a la fila a las 0
Cliente 8 ingresa a la fila a las 0
Cliente 9 ingresa a la fila a las 0
Cliente 10 ingresa a la fila a las 0
Cliente 1 es atendido a las 0
Cliente 2 es atendido a las 0
Cliente 3 es atendido a las 0
Cliente 4 es atendido a las 0
Cliente 5 es atendido a las 0
Atencion finalizada para el cliente 1 a las 60
Cliente 1 termina su atención a las 60
Cliente 6 es atendido a las 60
Cliente 11 ingresa a la fila a las 70
Cliente 12 ingresa a la fila a las 172
Atencion finalizada para el cliente 3 a las 263
Cliente 3 termina su atención a las 263
Cliente 7 es atendido a las 263
Atencion finalizada para el cliente 5 a las 270
Cliente 5 termina su atención a las 270
Cliente 8 es atendido a las 270
Cliente 13 ing